<a href="https://colab.research.google.com/github/zadwan/Graduate_Admissions/blob/master/Zeena_Adwan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### **1. Read in the three csv files as data frames. Combine all three data frames using admission_id into one named all_df, retaining all rows from Admissions.csv.**

In [ ]:
import pandas as pd

In [ ]:
# Reading the 3 csv files.
admission = pd.read_csv("Admissions.csv")
lab = pd.read_csv("Lab.csv")
transfusions = pd.read_csv("Transfusions.csv")

In [ ]:
# Displaying the first 5 rows in each file.
display(admission.head())
display(lab.head())
display(transfusions.head())

In [ ]:
# Dimensions of each file.
print(admission.shape)
print(lab.shape)
print(transfusions.shape)

In [ ]:
# Merging all 3 files into one dataframe (all_df).
all_df = admission.set_index('admission_id').join(lab.set_index('admission_id')).join(transfusions.set_index('admission_id'))

In [ ]:
# Shape and the first 5 rows of the new merged dataframe.
print(f'Shape of merged DataFrame: {all_df.shape}')
display(all_df.head())

#### **2. Impute the missing age and charlson_comorbidity_index values in any way you see fit. Briefly explain how the imputation method(s) might affect a statistical model that uses these variables as predictors.**

In [ ]:
# Info about each column to see the columns that have null values.
all_df.info()

In [ ]:
# Statistics of the numerical columns.
all_df.describe()

In [ ]:
# Filling the missing values in the age column with the mean the range of
# the values is compact. (filling with the median might also work but the mode isn't
# as practical here). Also, dropping the rows that has missing age values is fine.
all_df.age.fillna(all_df.age.mean(), inplace=True)

In [ ]:
# Invistegating the unique values of the charlson_comorbidity_index.
all_df.charlson_comorbidity_index.unique()

In [ ]:
# Replacing '2+' with '2' to convert the string numbers into integer numbers.
all_df.charlson_comorbidity_index.replace('2+', '2', inplace=True)
all_df.charlson_comorbidity_index.unique()

In [ ]:
# Filling the missing cells of "charlson_comorbidity_index" with the mean value=1.
all_df.charlson_comorbidity_index.fillna('1', inplace=True)
all_df.charlson_comorbidity_index.unique()

#### **3. Determine if there is a significant difference in age between patients who had an rbc_transfusion and patients that did not. Determine if there is a significant difference in sex between these groups.**

**A)** For age:  
I will assume a p-value threshold=0.05 (5%) and the following hypotheses:  

**Null Hypothesis:** Patients who had an rbc_transfusion are the same average age as people who didn't.  
**Alternative Hypothesis:** Patients who had an rbc_transfusion have different average age than people who didn't.

In [ ]:
# Investigating the unique values of "rbc_transfusion".
all_df.rbc_transfusion.unique()

In [ ]:
# Grouping the people that have "rbc_transfusion" and those who don't.
A_age = all_df[all_df['rbc_transfusion']==True]['age'].values.tolist()
B_age = all_df[all_df['rbc_transfusion']==False]['age'].values.tolist()

In [ ]:
# Performing T-test for the means of two independent samples of scores.
from scipy import stats
t_check = stats.ttest_ind(A_age, B_age)
print(f"P-value= {t_check[1]}")

Since the p-value of the test < 0.05, we reject the null hypothesis of equal averages. (There is a significant difference between the average age of people who had an rbc_transfusion and those who didn't).

**B)** For sex:  
I will assume a p-value threshold=0.05 (5%) and the following hypotheses:  

**Null Hypothesis:** Patients who had an rbc_transfusion are the same sex as people who didn't.  
**Alternative Hypothesis:** Patients who had an rbc_transfusion have different sex than people who didn't.

In [ ]:
all_df.sex.unique()

In [ ]:
all_df['sex'].replace(['F','M'],[0,1],inplace=True)

In [ ]:
A_sex = all_df[all_df['rbc_transfusion']==True]['sex'].values.tolist()
B_sex = all_df[all_df['rbc_transfusion']==False]['sex'].values.tolist()

In [ ]:
from scipy import stats
t_check = stats.ttest_ind(A_sex, B_sex)
print(f"P-value = {t_check[1]}")

Since the p-value of the test > 0.05, we failed to reject the null hypothesis of equal averages. (There is no significant difference between the sex of people who had an rbc_transfusion and those who didn't).

#### 4. Fit a linear regression model using the result_value of the “Platelet Count” lab tests as the dependent variable and age, sex, and hospital as the independent variables. Interpret the results.

In [ ]:
# Getting the rows that has test_name = "Platelet Count".
platelet_df = all_df[all_df.test_name == "Platelet Count"]

In [ ]:
# Statistics of the result_value.
platelet_df.result_value.describe()

In [ ]:
# Selecting the wanted columns.
reg_df = platelet_df[['age', 'sex', 'hospital']]
reg_df.head()

In [ ]:
reg_df.hospital.unique()

In [ ]:
# Performing one-hot-encoding on the "hospital" column.
X = pd.concat([reg_df, pd.get_dummies(reg_df.hospital)], axis=1)

In [ ]:
X.drop(['hospital'], axis=1, inplace=True)

In [ ]:
X.head()

In [ ]:
y = platelet_df.result_value

In [ ]:
print(X.shape)
print(y.shape)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [ ]:
# Splitting into train and test sets (80% to 20%).
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

In [ ]:
# Training the model.
reg = LinearRegression().fit(X_train, y_train)

In [ ]:
# Testing the model.
print(f"R^2 score: {reg.score(X_test, y_test)}")

The R^2 is too low which means there is no stron correlation between the dependent variable (result_value) and the independent variables (age, sex, and hospital).

#### **5. Create a plot that demonstrates the relationship between length_of_stay (discharge date and time minus admission date and time) and charlson_comorbidity_index. Briefly describe how you would change the visualization to add age as a third variable.**

# New Section

In [ ]:
all_df.head()

In [ ]:
adm = pd.to_datetime(all_df['admission_date'] + ' ' + all_df['admission_time'])
dis = pd.to_datetime(all_df['discharge_date'] + ' ' + all_df['discharge_time'])

In [ ]:
length_of_stay = (dis - adm).dt.components['minutes'].dropna()

In [ ]:
all_df.charlson_comorbidity_index

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.DataFrame({'time':length_of_stay,'data':len(all_df.charlson_comorbidity_index)})
df = df.set_index('time')
df.plot(yticks=[0,1],ylim=[-0.1,1.1])

6. **Fit a statistical model of your choosing to predict the result_value of the “Hemoglobin” lab tests and evaluate its performance. Explain your choices and how you might improve the model if you had to iterate on it. What other clinical variables would you be interested in adding to the model (they don’t have to be in the provided datasets)?**

In [ ]:
hemoglobin_df = all_df[all_df.test_name == "Hemoglobin"]

In [ ]:
hemoglobin_df

In [ ]:
# Converting True and False data to 0 and 1.
hemoglobin_df.charlson_comorbidity_index = hemoglobin_df.charlson_comorbidity_index.astype(int)

In [ ]:
# Investigating the correlation between the numeric columns and the "result_value" column.
import seaborn as sns
import matplotlib.pyplot as plt

corr = hemoglobin_df.corr()[['result_value']]
sns.heatmap(corr, cmap="YlGnBu", annot=True)
plt.show()

In [ ]:
# Selecting the 2 highest correlated columns.
X_hemo = hemoglobin_df[['sex', 'charlson_comorbidity_index']]

In [ ]:
y_hemo = hemoglobin_df.result_value

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_hemo, y_hemo, test_size=0.20, random_state=0)

In [ ]:
reg = LinearRegression().fit(X_train, y_train)
print(f"R^2 score: {reg.score(X_test, y_test)}")

The R^2 is too low which means there is no stron correlation between the dependent variable (result_value) and the independent variables (sex, and charlson_comorbidity_index).